## To do: explain here

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from fremontdropbox import get_dropbox_location
dbx = get_dropbox_location()

data_path = dbx + '/Private Structured data collection'

In [2]:
import geopandas as gpd
from keplergl import KeplerGl
import pandas as pd
import networkx as nx


## 1. Load Aimsun network

In [3]:
def read_gdf(path):
    gdf = gpd.GeoDataFrame.from_file(path)
    gdf = gdf.to_crs(epsg=4326)
    return gdf

Aimsun_nodes = read_gdf(data_path + "/Data processing/Raw/Network/Aimsun/nodes.shp")
Aimsun_sections = read_gdf(data_path + "/Data processing/Raw/Network/Aimsun/sections.shp")

In [4]:
print("Number of sections: " + str(Aimsun_sections.id.count()))
print("Number of nodes: " + str(Aimsun_nodes.id.count()))

Number of sections: 5626
Number of nodes: 2013


## To do: change NaN vertices to new vertices

In [5]:
# remove NaN vertices (to do)
edges_topo = Aimsun_sections # [['id', 'fnode', 'tnode', 'capacity', 'speed']]
edges_topo = edges_topo[edges_topo['fnode'].notna()]
edges_topo = edges_topo[edges_topo['tnode'].notna()]
edges_topo[['id', 'fnode', 'tnode', 'capacity', 'speed']] = edges_topo[['id', 'fnode', 'tnode', 'capacity', 'speed']].astype(int)



## 2. Load Aimsun output data on Aimsun road section

## To do: 
1. use AimsunAnalyzer.get_link_travel_time() from `Aimsun Microsimulation SQLite Output Database Analyzer` here
2. Set travel time as the default one if there is no vehicle on a link


## To do Zixuan and Yanda: make sure the following cell works

In [6]:
import analyser_utils as au

data_folder = os.path.join(dbx, 'Private Structured data collection')
sql_folder = os.path.join(data_folder, 'Aimsun','Outputs')
database = os.path.join(sql_folder, "2019_latest_0410.sqlite")
analyzer = au.AimsunAnalyzer(database, "sqlite")

# edges_topo.to_csv(sql_folder + '/test_pandas_to_add_travel_time.csv')
# edge_weighted = pd.read_csv(sql_folder + '/test_pandas_to_add_travel_time.csv')


=====Connection Established.=====
Simulation starts at 14h0min
=====Model Information Loaded.=====
=====Simulation Data Loaded.=====


In [12]:
edge_weighted = edges_topo.copy()
edge_weighted['travel time 2pm'] = edge_weighted['id'].apply(lambda x: analyzer.get_link_travel_time(x, "14:00"))
display(edge_weighted.head(3))


,id,eid,name,nb_lanes,speed,capacity,rd_type,func_class,fnode,tnode,geometry,travel time 2pm
0,242,242,None,1,120,2100,175.0,1,9845,9923,"LINESTRING (-121.92244 37.49593, -121.92242 37...",16.965624
1,243,243,None,3,104,6300,175.0,1,9852,9848,"LINESTRING (-121.92313 37.49526, -121.92173 37...",6.417711
2,244,244,None,3,104,6300,175.0,1,9850,9852,"LINESTRING (-121.92352 37.49561, -121.92313 37...",1.827915


In [13]:
print("Number of Aimsun sections: " + str(Aimsun_sections.id.count()))
print("Number of sections used (after dropna in Aimsun section): " + str(edge_weighted.id.count()))
print("Number of sections with travel time after adding simulation values: " + str(edge_weighted['travel time 2pm'].count()))



Number of Aimsun sections: 5626
Number of sections used (after dropna in Aimsun section): 4318
Number of sections with travel time after adding simulation values: 4318


## Computing isochrones with Networkx ego_graph

In [27]:

from shapely.geometry import Polygon
from shapely.ops import unary_union

def isochrone(dataframe, node_id, r):
    """
    To do
    """
    graph = nx.from_pandas_edgelist(dataframe, 'fnode', 'tnode', ['geometry', 'speed', 'capacity', 'travel time'])
    graph_within_r = nx.ego_graph(graph, node_id, radius=r, distance='travelTime')
    graph_df = nx.to_pandas_edgelist(graph_within_r)
    graph_gdf = gpd.GeoDataFrame(graph_df)
    return graph_gdf

def isochrones(edge_weighted, within_times, node_ids, buffer_size=0.0005):
    """
    To do
    """
    geometries = []
    for t in within_times:
        isochrone_result = Polygon()
        for nd in node_ids:
            graph_gdf = isochrone(edge_weighted, nd, t)
            isochrone_tmp = graph_gdf.geometry.buffer(buffer_size).unary_union
            isochrone_result = unary_union([isochrone_result, isochrone_tmp])
#             gpd.overlay(isochrone_result, isochrone_tmp, how='union') 
        geometries.append(isochrone_result)
    # print(gdf.geometry.buffer(0.0005).unary_union)
    isochrone_gdf = gpd.GeoDataFrame({'Travel time': within_times, 'geometry': geometries})
    return isochrone_gdf

def isochrones_with_time(edges_param, analyzer, time_interval, within_times, node_ids, buffer_size=0.0005):
    """
    To do
    """
    edges = edges_param.copy()
    edges['travel time'] = edges.id.apply(lambda x: analyzer.get_link_travel_time(x, time_interval))
    return isochrones(edges, within_times, node_ids, buffer_size)

## to do: try this code once it Aimsun Analyzer is a script
# time_interval = "14:00" # to do
# database = os.path.join(sql_folder, "2019_latest_0410.sqlite")
# analyzer = AimsunAnalyzer(database, "sqlite")
# test = isochrones_with_time(edges_topo, analyzer, interval_time, within_times, node_ids, buffer_size=0.0005)
    
node_ids = [15043, 9845]
times = [5*(i+1) for i in range(5)]
isochrone_gdf_2pm = isochrones_with_time(edge_weighted, analyzer, "14:00", times, node_ids, buffer_size=0.001)
print("half done")
isochrone_gdf_6pm = isochrones_with_time(edge_weighted, analyzer, "18:00", times, node_ids, buffer_size=0.001)


# print(isochrone_gdf)


half done


In [31]:
map_1 = KeplerGl(height=1000)
map_1.add_data(data=isochrone_gdf_2pm, name = "Isochrones 2pm")
map_1.add_data(data=isochrone_gdf_2pm[isochrone_gdf_2pm['Travel time']==10], name = "Isochrone 2 pm within 1 minute")

map_1.add_data(data=isochrone_gdf_6pm, name = "Isochrones 6pm")
map_1.add_data(data=isochrone_gdf_6pm[isochrone_gdf_6pm['Travel time']==15], name = "Isochrone 6pm within 1 minute")

# map_1.add_data(data=Aimsun_sections_simulation[Aimsun_sections_simulation.travelTime.notna()], name = "Sections with traffic")
map_1

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'Isochrones 2pm': {'index': [0, 1, 2, 3, 4], 'columns': ['Travel time', 'geometry'], 'data': [[…

## 3. Accessibility computation

## To do:
1. Load Google maps POIs
2. Merge POIs to nodes (see https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.KDTree.html)
3. Compute number of POIs accessible from set of points in 5/10/... minutes (using previous code and geopandas sjoin function)
4. Use pandana to compute some other accessibility indexes

## Appendix: visual rendering

In [ ]:
## Side work: plot sections that are not used

Aimsun_sections_simulation = pd.merge(Aimsun_sections,
                           vehSectTraj_temp,
                           left_on='id',
                           right_on='sectionId',
                           how='left',
                           sort=True)

map_1 = KeplerGl(height=1000)
map_1.add_data(data=Aimsun_sections_simulation[Aimsun_sections_simulation.travelTime.isnull()], name = "Sections without traffic")
map_1.add_data(data=Aimsun_sections_simulation[Aimsun_sections_simulation.travelTime.notna()], name = "Sections with traffic")
map_1

## The following is old

In [ ]:
# ## This should be removed once the function isochrones_with_time works
# # load Aimsun output data
# vehSectTraj = pd.read_csv(data_path + '/Aimsun/Outputs/vehSectTrajectory.csv')

# # to do here: only get data at a specific time step

# max_tt = vehSectTraj.travelTime.max()
# mean_tt = vehSectTraj.travelTime.mean()

# print(max_tt)
# print(mean_tt)

# # pivot veh sect traj
# vehSectTraj_temp = vehSectTraj.groupby("sectionId").mean()

# # merge it on Aimsun sections
# edge_weighted = pd.merge(edges_topo,
#                            vehSectTraj_temp,
#                            left_on='id',
#                            right_on='sectionId',
#                            how='left',
#                            sort=True)

# edge_weighted.travelTime.fillna(mean_tt, inplace = True)